# 1. Loading the dataset

In [12]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [13]:
file_path = "./datasets/train.txt"
train = pd.read_csv(file_path, sep='\t', header=None, names=['LABEL', 'REVIEW'])
train

,LABEL,REVIEW
0,TRUTHFULPOSITIVE,The sheraton was a wonderful hotel! When me an...
1,TRUTHFULPOSITIVE,We stayed at the Omni between Christmas and Ne...
2,DECEPTIVENEGATIVE,I was REALLY looking forward to a nice relaxin...
3,TRUTHFULNEGATIVE,"First let me say, I try not to be too critical..."
4,DECEPTIVENEGATIVE,The Ambassador East Hotel is a terrible place ...
...,...,...
1395,TRUTHFULNEGATIVE,I stayed here for 5 nights last summer. I book...
1396,TRUTHFULPOSITIVE,Stayed here for 3 nights for a Bridgestone/Fir...
1397,TRUTHFULNEGATIVE,I am staying here now and actually am compelle...
1398,TRUTHFULNEGATIVE,We stayed at this hotel with our two teenage d...


In [14]:
file_path = "./datasets/test_just_reviews.txt"
test = pd.read_csv(file_path, sep='\t', header=None, names=['REVIEW'])
test

,REVIEW
0,My family and I stayed here while we were visi...
1,WARNING! My stay at the Talbott Hotel will go ...
2,I recently stayed at the Hard Rock Hotel in Ch...
3,O.M.G best hotel ever ! i've stayed at various...
4,We became an Ambassador member just before spe...
...,...
195,The Millennium Knickerbocker Hotel has seen be...
196,We got a spanking deal at this hotel for $99 a...
197,Just back from a business trip. The Homewood i...
198,I have just returned from a lovely shopping tr...


In [15]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train['REVIEW'].values)

X_train = tokenizer.texts_to_sequences(train['REVIEW'].values)
X_test = tokenizer.texts_to_sequences(test['REVIEW'].values)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

In [16]:
print('Original text:')
print('\t' + train['REVIEW'].values[2])
print('Tokenized vector:')
print('\t', X_train[2])

print('Tokens of the first words:')
for word in ['I', 'was', 'REALLY', 'looking']:
    print('\t{}: {}'.format(word, tokenizer.word_index[word.lower()]))

Original text:
	I was REALLY looking forward to a nice relaxing stay at the end of a long vacation, but unfortunately that was not to be had. From the moment we arrived at the Omni, the staff was belligerent and extremely rude. They had lost our reservation and refused to give us the rate we had booked before. If we weren't so tired, we would've gone to a different hotel right then, but hindsight is 20/20. After we FINALLY got checked in after being made to wait needlessly for 45 minutes in the lobby, we decided to go down to the pool, which was tiny and kind of dirty, so we had walked all the way down there in our suits for nothing. The internet access was really slow. I will NEVER stay here again. Save yourself the headache and book somewhere else!
Tokenized vector:
	 [5, 6, 96, 188, 617, 3, 4, 63, 510, 29, 14, 1, 433, 9, 4, 198, 426, 25, 526, 17, 6, 19, 3, 33, 22, 35, 1, 690, 10, 130, 14, 1, 311, 1, 42, 6, 2, 249, 220, 26, 22, 691, 27, 169, 2, 1080, 3, 328, 52, 1, 282, 10, 22, 149, 

In [17]:
from keras_preprocessing.sequence import pad_sequences

maxlen = 400

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[0, :])

[   1  419    6    4  173    8   36   43    2   15 2336 2664    7   10
   20   96  525   40   10  251    3  172    4  600 2665   10 1010  195
    3   56   51    1  158   44  363    3 1538   11    5  293    3  172
   13  223   18   43    1   41    6   39    2   21    6  656   45    9
    1 1895   53   19    1 1895    8 2666  161   47    7   26   22    4
   96   63  135  604   18  190  126    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [18]:
from keras.models import Sequential
from keras import layers

embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 400, 50)           458900    
                                                                 
 global_max_pooling1d_6 (Glo  (None, 50)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_12 (Dense)            (None, 10)                510       
                                                                 
 dense_13 (Dense)            (None, 4)                 44        
                                                                 
Total params: 459,454
Trainable params: 459,454
Non-trainable params: 0
_________________________________________________________________


In [19]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

encoder = LabelEncoder()
labels = encoder.fit_transform(train['LABEL'].values)
labels = to_categorical(labels)
labels

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [20]:
from sklearn.model_selection import KFold, StratifiedKFold

kf = KFold(n_splits = 5)
train_accs = []
val_accs = []
for train_index, val_index in kf.split(X_train, labels):
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size, 
                            output_dim=embedding_dim, 
                            input_length=maxlen))
    model.add(layers.GlobalMaxPool1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(4, activation='softmax'))
    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    training_data = X_train[train_index]
    training_labels = labels[train_index]
    validation_data = X_train[val_index]
    validation_labels = labels[val_index]
    model.fit(training_data, training_labels, epochs=20, validation_data=(validation_data, validation_labels), verbose=True, batch_size=10)
    _, accuracy = model.evaluate(training_data, training_labels, verbose=False)
    train_accs.append(accuracy)
    _, accuracy = model.evaluate(validation_data, validation_labels, verbose=False)
    val_accs.append(accuracy)
print(train_accs)
print(val_accs)

Epoch 1/20


112/112 [==============================] - 11s 33ms/step - loss: 1.3720 - accuracy: 0.3187 - val_loss: 1.3455 - val_accuracy: 0.4500
Epoch 2/20
112/112 [==============================] - 3s 24ms/step - loss: 1.2847 - accuracy: 0.4741 - val_loss: 1.2210 - val_accuracy: 0.4786
Epoch 3/20
112/112 [==============================] - 3s 24ms/step - loss: 1.0875 - accuracy: 0.5107 - val_loss: 1.0248 - val_accuracy: 0.5393
Epoch 4/20
112/112 [==============================] - 3s 23ms/step - loss: 0.8754 - accuracy: 0.6268 - val_loss: 0.8856 - val_accuracy: 0.6643
Epoch 5/20
112/112 [==============================] - 3s 24ms/step - loss: 0.7153 - accuracy: 0.7884 - val_loss: 0.7842 - val_accuracy: 0.7250
Epoch 6/20
112/112 [==============================] - 3s 23ms/step - loss: 0.5732 - accuracy: 0.8696 - val_loss: 0.7075 - val_accuracy: 0.7500
Epoch 7/20
112/112 [==============================] - 3s 24ms/step - loss: 0.4394 - accuracy: 0.9187 - val_loss: 0.6464 - val_accuracy: 0.7464
Epoch 8/2

In [23]:
np.mean(val_accs)

0.7464285612106323

# Creating test output

In [24]:
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)
predicted_class_labels = encoder.inverse_transform(predicted_labels)

7/7 [==============================] - 0s 4ms/step


In [ ]:
f = open("results/NN.txt", "w")
for s in predicted_class_labels:
    f.write(s + '\n')
f.close()